In [ ]:
# Import modules
import cv2
import sys
import os
import matplotlib.pyplot as plt
from IPython.display import HTML
import urllib

video_input_file_name = "/Users/samuel/realtime_obj_detection/race_car.mp4"

def drawRectangle(frame, bbox):
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)

def displayRectangle(frame, bbox):
    plt.figure(figsize=(20, 10))
    frameCopy = frame.copy()
    drawRectangle(frameCopy, bbox)
    frameCopy = cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB) 
    plt.imshow(frameCopy)
    plt.axis('off')
    plt.show()

def drawText(frame, txt, location, color=(50, 170, 50)):
    cv2.putText(frame, txt, location, cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)

if not os.path.isfile('goturn.prototxt') or not os.path.isfile('goturn.caffemodel'):
    print("Downloading GOTURN model zip file")
    urllib.request.urlretrieve('https://www.dropbox.com/sh/77frbrkmf9ojfm6/AACgY7-wSfj-LIyYcOgUSZ0Ua?dl=1', 'GOTURN.zip')


    os.system('tar -xvf GOTURN.zip') 

    os.remove('GOTURN.zip')

tracker_types = ['MIL', 'GOTURN',]

tracker_type = tracker_types[0]

if tracker_type == 'MIL':
    tracker = cv2.TrackerMIL_create()
    

elif tracker_type == 'GOTURN':
    tracker = cv2.TrackerGOTURN_create()
video = cv2.VideoCapture(video_input_file_name)

if not video.isOpened():
    print("Error: Could not open video.")
    sys.exit()

ok, frame = video.read()
if not ok:
    print("Error: Could not read video file.")
    sys.exit()

width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

video_output_file_name = f'race_car-{tracker_type}.mp4'
video_out = cv2.VideoWriter(video_output_file_name, cv2.VideoWriter_fourcc(*'avc1'), 10, (width, height))

bbox = (1300, 405, 160, 120)

displayRectangle(frame, bbox)
ok = tracker.init(frame, bbox)

while True:
    ok, frame = video.read()
    if not ok:
        break

    timer = cv2.getTickCount()

    ok, bbox = tracker.update(frame)

    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

    if ok:
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (0, 255, 0), 2, 1)
    else:
        drawText(frame, 'Tracking failure detected', (80, 140), (0, 0, 255))

    drawText(frame, "Tracker: " + tracker_type, (80, 60))
    drawText(frame, "FPS: " + str(int(fps)), (80, 100))

    video_out.write(frame)

    cv2.imshow('Tracking', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
video_out.release()
cv2.destroyAllWindows()
